In [1]:
import sys, os, gzip, shutil
import datetime as dt
import pandas as pd
import numpy as np
import holidays

In [2]:
PEMSDIR = r'C:\Users\xzh263\Dropbox (KTC)\SFCTA CMP\2021 CMP\PeMS'
OUTDIR = PEMSDIR

data_type = 'station_meta'
district = 4
ca_holidays = holidays.UnitedStates(state='CA')

In [3]:
obs_pct_min = 20  # Minimum observation percentage requirement
sample_pct_min = 50 # Minimum sample percentage requirement

In [4]:
monitor_loc = pd.read_csv(os.path.join(OUTDIR,'pems_monitoring_locations.csv'))

In [5]:
def get_dir(base, year=2021, data_type='station_hour', district=4):
    if data_type in ['station_hour','station_5min','station_meta']:
        return os.path.join(PEMSDIR,'D{}_Data_{}\{}'.format(district,year,data_type))
def get_columns(data_type, num_cols):
    if data_type == 'station_meta':
        columns = ['station','route','dir','district','county','city','state_postmile','abs_postmile','latitude','longitude',
                   'length','type','lanes','name','user_id_1','user_id_2','user_id_3','user_id_4']
    if data_type == 'station_hour':
        columns = ['timestamp', 'station', 'district', 'route', 'dir', 'lane_type', 'station_length',
                   'samples', 'obs_pct', 'total_flow', 'avg_occupancy', 'avg_speed',
                   'delay_35','delay_40','delay_45','delay_50','delay_55','delay_60']
        for i in range(0, int((num_cols - 18) / 3)):
            columns += [f'lane_{i}_flow',
                        f'lane_{i}_avg_occ',
                        f'lane_{i}_avg_speed',
                       ]
    if data_type == 'station_5min':
        columns = ['timestamp', 'station', 'district', 'route', 'dir', 'lane_type', 'station_length',
                   'samples', 'obs_pct', 'total_flow', 'avg_occupancy', 'avg_speed']
        for i in range(0, int((num_cols - 12) / 5)):
            columns += [f'lane_{i}_samples',
                        f'lane_{i}_flow',
                        f'lane_{i}_avg_occ',
                        f'lane_{i}_avg_speed',
                        f'lane_{i}_avg_obs',
                       ]
    return columns

In [6]:
unzip = False
source = 'gz' # or 'zip','text','txt'
save_h5 = True
data_type = 'station_hour'
sep = ','

for year in np.arange(2021,2022):
    year_dfs = []
    path = get_dir(PEMSDIR, year, data_type, district)
    outpath = os.path.join(OUTDIR,'pems')
    contents = os.listdir(path)
    gzs = filter(lambda x: os.path.splitext(x)[1] == '.gz', contents)
    txts = filter(lambda x: os.path.splitext(x)[1] == '.txt', contents)

    if source == 'gz':
        files = gzs
        compression = 'gzip'
    else:
        files = txts
        compression = None

    header = 0 if data_type == 'station_meta' else None

    for f in files:
        print(f)
        try:
            df = pd.read_csv(os.path.join(path, f), 
                             sep=sep,
                             header=header, 
                             index_col=False, 
                             parse_dates=[0], 
                             infer_datetime_format=True,
                             compression=compression)
        except Exception as e:
            print(e)
            print('trying no quotechar...')
            try:
                df = pd.read_csv(os.path.join(path, f), 
                                 sep=sep,
                                 header=header, 
                                 index_col=False, 
                                 parse_dates=[0], 
                                 infer_datetime_format=True,
                                 quotechar=None,
                                 compression=compression)

            except Exception as e2:
                print(e2)
                continue

        try:
            df.columns = get_columns(data_type, len(df.columns))
        except Exception as e3:
            print(e3)
            continue
        if data_type == 'station_meta':
            y, m, d = f.replace('d{:02d}_text_meta_'.format(district),'').replace('.txt','').split('_')
            ts = dt.datetime(int(y), int(m), int(d))
            date = ts.date()
            df['timestamp'] = ts
            df['date'] = date
            df['year'] = y
            df['month'] = m
            df['day'] = d
            meta.append(df)
        elif data_type == 'station_hour':
            df['date'] = df['timestamp'].map(lambda x: x.date())
            df['year'] = df['timestamp'].map(lambda x: x.year)
            df['month'] = df['timestamp'].map(lambda x: x.month)
            df['day'] = df['timestamp'].map(lambda x: x.day)
            df['hour'] = df['timestamp'].map(lambda x: x.hour)
            df['day_of_week'] = df['timestamp'].map(lambda x: x.weekday())
            df['is_holiday'] = df['timestamp'].map(lambda x: x.date() in ca_holidays)
            year_dfs.append(df)
            
    y = pd.concat(year_dfs)
    try:
        y.to_hdf(os.path.join(OUTDIR,'pems_station_hour_{}.h5'.format(year)), 'data')
    except Exception as e:
        print(e)

d04_text_station_hour_2021_01.txt.gz
d04_text_station_hour_2021_02.txt.gz
d04_text_station_hour_2021_03.txt.gz


C:\Users\xzh263\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->Index(['dir', 'lane_type', 'date'], dtype='object')]

  encoding=encoding,


In [7]:
stations = pd.read_csv(os.path.join(OUTDIR,'D4_Data_2021\station_meta\d04_text_meta_2021_03_19.txt'), delimiter = "\t")

In [8]:
#PeMS stations in SF
sf_stations = stations[stations['County']==75]

In [9]:
sf_counts = y[y['station'].isin(sf_stations['ID'])]

In [10]:
sf_counts['daytype'] = np.where(sf_counts['day_of_week']<=4, 'Weekdays', np.where(sf_counts['day_of_week']==5, 'Saturday', 'Sunday'))

C:\Users\xzh263\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
sf_counts = sf_counts.merge(sf_stations[['ID', 'Lanes']], left_on='station', right_on='ID', how='left')
sf_counts['sample_pct'] = 100 * sf_counts['samples'] / 60 / 2 / sf_counts['Lanes'] 

In [12]:
groupby_cols = ['route','dir','lane_type','station', 'daytype', 'hour']
agg_args = {'total_flow':['mean','count','std']}

df = sf_counts.loc[(sf_counts['day_of_week']!=0) & (sf_counts['day_of_week']!=4) & ~sf_counts['is_holiday']]

df_agg = df.loc[(df['obs_pct']>=obs_pct_min) & (df['sample_pct']>=sample_pct_min)].groupby(groupby_cols).agg(agg_args).reset_index()
df_agg.columns = groupby_cols + ['flow_avg', 'flow_count', 'flow_std']

In [13]:
df_agg = df_agg.merge(monitor_loc, on='station')

In [14]:
groupby_cols = ['location','direction', 'daytype', 'hour']

df_out = df_agg.groupby(groupby_cols).flow_avg.mean().reset_index()
df_out.columns = groupby_cols + ['flow_avg']
df_out['flow_avg'] = df_out['flow_avg'].round()

In [15]:
df_out.to_csv(os.path.join(OUTDIR, 'cmp2021_pems_volumes.csv'), index=False)